In [ ]:
# -*- coding: utf-8 -*-

"""
{Description}

MIT License

Copyright (c) 2021, Fantasy-Sidelines

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY +OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

Sportradar API details and documentation: https://developer.sportradar.com/docs
MySportsFeeds API details and documentation: https://www.mysportsfeeds.com/data-feeds/api-docs/
www.pro-football-reference.come and documentation: https://www.sports-reference.com/termsofuse.html
"""

import pyodbc
import os
import pandas as pd
import numpy as np
from gspread_pandas import Spread
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.engine import URL
from dotenv import load_dotenv

__author__ = "Tim Cuddeback"
__copyright__ = "Copyright (c) 2021, Fantasy-Sidelines"
__credits__ = ["Tim Cuddeback", "Sportradar API", "Fantasy Sharks", "MySportsFeeds API"]
__license__ = "MIT License"
__version__ = "1.1.0"
__maintainer__ = "Tim Cuddeback"
__email__ = "cuddebtj@gmail.com"
__status__ = "Dev"

from sql_upload import *

In [ ]:
load_dotenv()
sql_driver = os.getenv("sql_driver")
# check IP address for connection timeout errors
sql_server_16 = os.getenv("sql_server_16")
sql_server_17 = os.getenv("sql_server_17")
sql_database = os.getenv("sql_database")
sql_username = os.getenv("sql_username")
sql_password = os.getenv("sql_password")
api_key = os.getenv("sportradar_apikey")

year = [2020, 2019, 2018, 2017, 2016, 2015]

connection_string = ("DRIVER="+sql_driver
                     +";SERVER="+sql_server_16
                     +";DATABASE="+sql_database
                     +";UID="+sql_username
                     +";PWD="+sql_password)
# cxn = pyodbc.connect(connection_string)
connection_url = URL.create("mssql+pyodbc", 
                            query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
conn = engine.connect()

inspector = inspect(engine)
print(inspector.get_table_names())

In [ ]:
# schedule_stats_api_sql(api_key, year, engine)
# snaps(y1, y2, engine)
# injuries(y1, y2, engine)
# player_table(engine)
# game_table(engine)
# season_table(engine)
# week_table(engine)
# team_table(engine)
# venue_table(engine)

# spread = Spread("PracticeParticipation")

# weeks = list(range(1,22))
# seasons = list(range(2016, 2022))
# teams_id = ["Arizona Cardinals", "Atlanta Falcons", "Baltimore Ravens", "Buffalo Bills", 
#             "Carolina Panthers", "Chicago Bears", "Cincinnati Bengals", "Cleveland Browns", 
#             "Dallas Cowboys", "Denver Broncos", "Detroit Lions", "Green Bay Packers", 
#             "Houston Texans", "Indianapolis Colts", "Jacksonville Jaguars", "Kansas City Chiefs", 
#             "Los Angeles Rams", "San Diego Chargers", "Miami Dolphins", "Minnesota Vikings", "New England Patriots", 
#             "New Orleans Saints", "New York Giants", "New York Jets", "Oakland Raiders", "Las Vegas Raiders",
#             "Philadelphia Eagles", "Pittsburgh Steelers", "Los Angeles Chargers", "San Francisco 49ers", 
#             "Seattle Seahawks", "Tampa Bay Buccaneers", "Tennessee Titans", "Washington Redskins", "Washington Football Team"]

# practice_par_df = pd.DataFrame()

# for season in seasons:
    
#     for week in weeks:
        
#         try:
#             sh = f"{str(season)}w{str(week)}"
#             sheet = spread.sheet_to_df(header_rows=None, sheet=sh)
#             sheet = sheet.reset_index()

#             idx = [sheet["index"][sheet["index"] == d].index[0] for t in teams_id for d in sheet["index"] if t in d]
#             idx.append(len(sheet["index"]))
#             idx.sort()

#             final_df = pd.DataFrame()

#             for i in range(len(idx[:-1])):
                
#                 sheet.loc[idx[i]+2:idx[i+1]-1, "team"] = sheet["index"].loc[idx[i]]
                
#                 team = sheet.loc[idx[i]+1:idx[i+1]-1]
                
#                 columns = team.iloc[0].values.tolist()
#                 columns = columns[:-1]
#                 columns[1] = "injury"
#                 columns.append("team")
                
#                 team.columns = columns
#                 team = team[team["Player"] != "Player"]
                
#                 final_df = pd.concat([final_df, team])

#             final_df["season"] = season
#             final_df["week"] = week
            
#             cols = final_df.columns.tolist()
#             cols.remove("Player")
#             cols.remove("injury")
#             cols.remove("team")
#             cols.remove("week")
#             cols.remove("season")
            
#             final_df = pd.melt(final_df, 
#                                id_vars=["Player", "injury", "team", "season", "week"], 
#                                value_vars=cols, 
#                                var_name="date", 
#                                value_name="status", 
#                                ignore_index=True)
            
#             final_df = final_df.dropna(subset=["status"])
#             final_df[["player", "pos"]] = final_df["Player"].str.split(", ", 1, expand=True)
#             final_df[["last.date", "status", "home.away", "opp"]] = final_df["status"].str.split(" ", expand=True)
#             final_df = final_df.drop(["Player"], axis=1)
#             final_df["status"] = final_df["status"].fillna(value=final_df["last.date"])
#             final_df.sort_values(["team", "player"], inplace=True)
#             final_df["home.away"] = final_df.groupby("team")["home.away"].transform(lambda x: x.bfill().ffill())
#             final_df["opp"] = final_df.groupby("team")["opp"].transform(lambda x: x.bfill().ffill())
#             final_df[["day.name.abr", "date"]] = final_df["date"].str.split(" ", expand=True)
#             final_df["date"] = final_df["date"].replace("Status", np.nan)
#             final_df = final_df[["player", "pos", "team", 
#                                  "home.away", "opp", "season", 
#                                  "week", "date", "status", 
#                                  "injury", "day.name.abr"]]

#             spread.df_to_sheet(final_df, index=False, sheet=sh, replace=True)
#             practice_par_df = pd.concat([practice_par_df, sheet])
                
#         except Exception as e:
#             print(e)
#             print(sh)
#             break
            
# practice_par_df.to_sql("practiceParticipation", engine, if_exists="append", index=False, schema="dbo", chunksize=500)

In [ ]:
# start = "1/1/2015"
# end = "12/31/2025"
# calendar_df = pd.DataFrame(pd.date_range(start, end), columns=["date"])
# calendar_df["day.number"] = calendar_df["date"].dt.day
# calendar_df["day.abr"] = calendar_df["date"].dt.strftime("%a")
# calendar_df["day.name"] = calendar_df["date"].dt.strftime("%A")
# calendar_df["month.number"] = calendar_df["date"].dt.month
# calendar_df["month.abr"] = calendar_df["date"].dt.strftime("%b")
# calendar_df["month.name"] = calendar_df["date"].dt.strftime("%B")
# calendar_df["year"] = calendar_df["date"].dt.year
# calendar_df["week.number"] = calendar_df["date"].dt.isocalendar().week
# calendar_df["day.year.number"] = calendar_df["date"].dt.dayofyear
# calendar_df
# calendar_df.to_sql("calendarIDTable", engine, if_exists="replace", index=False, schema="dbo", chunksize=500)

In [ ]:
conn.close()
# cxn.close()